In [2]:
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

### Génération d'un TVAR(d)

In [4]:
def generation_TVAR(d, T, epsilon):
    
    """generation des coefficients du TVAR par les corrélations partielles de l'algorithme de Levinson"""
    
    #améliorer la génération des kapas avec des fonctionns à valeur de [1,d] dans [-1,1]^d
    kappa = 2 * np.random.rand(d, 1) - 1
    theta = np.matrix(np.zeros((d,t)))
    theta_inter = np.matrix(np.zeros((d,d)))
    X = epsilon
    
    for k in np.arange(d):
        theta[k,k] = kappa[k]
    
    for t in np.arange(T):
        for p in np.arange(1, d):
            for k in np.arange(1, p):
                theta_inter[k,p] = theta_inter[k,p-1] - kappa[p] * theta_inter[p-k,p-1]
        theta[:,t] = theta_inter[d,:].T
        X[T + t] = np.dot(X[T+t-d:T+t][::-1], theta[:,t]) + X[T + t]
        
    return X, theta

### Analyse spectrale

In [ ]:
from scipy.fftpack import fft

def dsp(theta, T, N = 512):
    dsp_array = np.matrix(np.zeros((N, T)))
    
    for t in np.arange(T):
        dsp_array[:,t] = np.matrix(1./abs(fft(theta[:,t].T, N)) ** 2).T
        
    return dsp_array

### Construction d'un estimateur en ligne

In [ ]:
from scipy.linalg import norm

def generation_est(X, d, T, mu):

    theta_est = np.matrix(np.zeros((d,T)))
    X_est = np.zeros(T)
    
    for k in (np.arange(T)): # on fait T itérations
        XX = X[T+k-d:T+k][::-1]
        
        if k==0:
            theta_est[:,k] = np.matrix(mu * X[T+k] * XX / (1 + mu * norm(XX) ** 2)).T 
        else:
            theta_est[:,k] = np.matrix(theta_est[:,k-1].T + mu * (X[T+k] - np.dot(XX, theta_est[:,k-1])) * XX / (1 + mu * norm(XX) ** 2)).T  

        X_est[k] = np.dot(XX, theta_est[:,k])
        
    return X_est, theta_est

### Estimation par aggregation